## Assignment 1

In [1]:
from google.colab import files
uploaded = files.upload()

Saving UIT-VSFC.zip to UIT-VSFC.zip


In [2]:
import zipfile
import io
with zipfile.ZipFile(io.BytesIO(uploaded['UIT-VSFC.zip']), 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [3]:
!pip3 install py_vncorenlp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4304 sha256=cc283b045778a195c54fcadff7fe365fa263f1eefff29d4b415ddae6610c9573
  Stored in directory: /root/.cache/pip/wheels/6d/2d/d6/158260bfd6820d144535857b80cc112bee5c3aa6d81b6dc049
Successfully built py_vncorenlp


In [4]:
!mkdir vncorenlp

In [5]:
import py_vncorenlp

py_vncorenlp.download_model(save_dir='/content/vncorenlp')
annotator = py_vncorenlp.VnCoreNLP(save_dir='/content/vncorenlp')

In [6]:
import json

def load_json_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

In [7]:
train_data = load_json_data('/content/UIT-VSFC/UIT-VSFC-train.json')
train_data[0]

{'sentence': 'slide giáo trình đầy đủ .',
 'sentiment': 'positive',
 'topic': 'training_program'}

In [8]:
def word_segment(data):
    for item in data:
        sentence = item['sentence']
        segmented = annotator.annotate_text(sentence)[0]
        annotated = []
        for word in segmented:
            wordForm = word['wordForm']
            annotated.append(wordForm)
        annotated = ' '.join(annotated)
        item['segmented'] = annotated
    return data

In [9]:
train_data = word_segment(train_data)
train_data[0]

{'sentence': 'slide giáo trình đầy đủ .',
 'sentiment': 'positive',
 'topic': 'training_program',
 'segmented': 'slide giáo_trình đầy_đủ .'}

In [10]:
test_data = word_segment(load_json_data('/content/UIT-VSFC/UIT-VSFC-test.json'))

## Assignment 2

In [11]:
!pip install transformers

In [12]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [13]:
def sentence_embedding(sentence):
    input_ids = tokenizer.encode(sentence, return_tensors='pt')
    with torch.no_grad():
        last_hidden_state = phobert(input_ids)[0]
        return last_hidden_state[0][1:-1].mean(dim=0).numpy()

In [14]:
import numpy as np
np.random.seed(42)
def dataset_embedding(data):
    embeddings, labels = [], []
    for item in data:
        embedded = sentence_embedding(item['segmented'])
        embeddings.append(embedded)
        labels.append(item['sentiment'])
    return np.array(embeddings), np.array(labels)

In [15]:
X_train, y_train = dataset_embedding(train_data)
X_test, y_test = dataset_embedding(test_data)

## Assignment 3

In [16]:
class GaussianMixtureModel:
    def __init__(self, n_components=3, max_iter=100, tol=1e-4, reg_covar=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
        self.reg_covar = reg_covar

    def _multivariate_normal(self, X, mean, cov):
        D = X.shape[1]
        diff = X - mean
        try:
            inv_cov = np.linalg.inv(cov)
        except np.linalg.LinAlgError:
            cov += self.reg_covar * np.eye(D)
            inv_cov = np.linalg.inv(cov)

        det_cov = np.linalg.det(cov)
        if det_cov <= 0:
            det_cov = 1e-12

        exponent = np.einsum('ij,jk,ik->i', diff, inv_cov, diff)
        norm_const = np.sqrt((2 * np.pi) ** D * det_cov)
        return np.exp(-0.5 * exponent) / norm_const

    def initialize(self, X):
        N, D = X.shape
        self.weights = np.ones(self.n_components) / self.n_components
        self.means = X[np.random.choice(N, self.n_components, replace=False)]
        cov_base = np.cov(X, rowvar=False) + self.reg_covar * np.eye(D)
        self.covs = np.array([cov_base.copy() for _ in range(self.n_components)])

    def e_step(self, X):
        N = X.shape[0]
        self.resp = np.zeros((N, self.n_components))
        for k in range(self.n_components):
            self.resp[:, k] = self.weights[k] * self._multivariate_normal(X, self.means[k], self.covs[k])
        self.resp /= np.clip(self.resp.sum(axis=1, keepdims=True), 1e-12, None)

    def m_step(self, X):
        N, D = X.shape
        Nk = self.resp.sum(axis=0)
        self.weights = Nk / N
        self.means = (self.resp.T @ X) / Nk[:, None]
        for k in range(self.n_components):
            diff = X - self.means[k]
            weighted_diff = self.resp[:, k][:, None] * diff
            cov_k = (weighted_diff.T @ diff) / Nk[k]
            self.covs[k] = cov_k + self.reg_covar * np.eye(D)

    def compute_log_likelihood(self, X):
        N = X.shape[0]
        log_likelihood = 0
        for i in range(N):
            prob = 0
            for k in range(self.n_components):
                p = self.weights[k] * self._multivariate_normal(X[i:i+1], self.means[k], self.covs[k])
                prob += p
            log_likelihood += np.log(prob + 1e-12)
        return log_likelihood

    def fit(self, X, verbose=False):
        self.initialize(X)
        prev_ll = -np.inf
        self.log_likelihoods = []

        for iteration in range(self.max_iter):
            self.e_step(X)
            self.m_step(X)
            ll = self.compute_log_likelihood(X)

            if verbose:
                print(f"Iter {iteration}, log-likelihood: {float(ll)}")

            if not np.isfinite(ll):
                print("⛔ Dừng sớm: Log-likelihood không ổn định (NaN hoặc Inf).")
                break

            self.log_likelihoods.append(ll)
            if np.abs(ll - prev_ll) < self.tol:
                if verbose:
                    print("✅ Hội tụ.")
                break
            prev_ll = ll

    def predict(self, X):
        N = X.shape[0]
        probs = np.zeros((N, self.n_components))
        for k in range(self.n_components):
            probs[:, k] = self.weights[k] * self._multivariate_normal(X, self.means[k], self.covs[k])
        return np.argmax(probs, axis=1)

In [17]:
def PCA_reduce(X, k):
    """
    Giảm chiều dữ liệu X xuống k chiều bằng PCA thuần NumPy.
    X: ndarray (N, D)
    k: số chiều muốn giữ lại
    """
    # 1. Chuẩn hóa dữ liệu (trừ trung bình)
    X_centered = X - np.mean(X, axis=0)

    # 2. Tính ma trận hiệp phương sai
    cov_matrix = np.cov(X_centered, rowvar=False)

    # 3. Phân rã thành trị riêng & vector riêng
    eig_vals, eig_vecs = np.linalg.eigh(cov_matrix)

    # 4. Sắp xếp giảm dần theo trị riêng
    sorted_idx = np.argsort(eig_vals)[::-1]
    eig_vecs = eig_vecs[:, sorted_idx]
    eig_vals = eig_vals[sorted_idx]

    # 5. Chọn k vector riêng đầu tiên
    components = eig_vecs[:, :k]

    # 6. Dựng lại dữ liệu giảm chiều
    X_reduced = np.dot(X_centered, components)
    return X_reduced

In [18]:
def map_clusters_to_labels(y_true, y_pred, n_components):
    mapping = {}
    for c in range(n_components):
        indices = np.where(y_pred == c)[0]
        if len(indices) == 0:
            continue
        true_labels = y_true[indices]
        unique_labels, counts = np.unique(true_labels, return_counts=True)
        most_common_label = unique_labels[np.argmax(counts)]
        mapping[c] = most_common_label
    y_pred_mapped = np.array([mapping[c] for c in y_pred])
    return y_pred_mapped, mapping

In [19]:
def evaluate(y_true, y_pred):
    labels = np.unique(y_true)
    metrics = {}
    for label in labels:
        tp = np.sum((y_true == label) & (y_pred == label))
        fp = np.sum((y_true != label) & (y_pred == label))
        fn = np.sum((y_true == label) & (y_pred != label))
        precision = tp / (tp + fp + 1e-12)
        recall = tp / (tp + fn + 1e-12)
        f1 = 2 * precision * recall / (precision + recall + 1e-12)
        metrics[label] = {"precision": precision, "recall": recall, "f1": f1}
    return metrics

In [20]:
X_train_reduced = PCA_reduce(X_train, k=78)
X_test_reduced = PCA_reduce(X_test, k=78)

In [21]:
gmm = GaussianMixtureModel(n_components=3, max_iter=200, tol=1e-3)
gmm.fit(X_train_reduced, verbose=True)

<ipython-input-16-823359031581>:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"Iter {iteration}, log-likelihood: {float(ll)}")


Iter 0, log-likelihood: 43503.751478775994
Iter 1, log-likelihood: 63481.778127143756
Iter 2, log-likelihood: 76632.63145176721
Iter 3, log-likelihood: 86881.6267389626
Iter 4, log-likelihood: 95671.72731742582
Iter 5, log-likelihood: 107354.25867007535
Iter 6, log-likelihood: 119504.16004730946
Iter 7, log-likelihood: 129163.17622812359
Iter 8, log-likelihood: 133949.32839365417
Iter 9, log-likelihood: 136620.10842841698
Iter 10, log-likelihood: 138482.32153760927
Iter 11, log-likelihood: 139260.16970390186
Iter 12, log-likelihood: 139680.51913116587
Iter 13, log-likelihood: 139913.45850170424
Iter 14, log-likelihood: 140096.15782226538
Iter 15, log-likelihood: 140302.99505391633
Iter 16, log-likelihood: 140414.01686992153
Iter 17, log-likelihood: 140496.30926913672
Iter 18, log-likelihood: 140569.69946390513
Iter 19, log-likelihood: 140631.07684226776
Iter 20, log-likelihood: 140655.6782528089
Iter 21, log-likelihood: 140685.85684475035
Iter 22, log-likelihood: 140690.92534485497
Ite

In [22]:
y_test_cluster = gmm.predict(X_test_reduced)
y_test_pred, cluster_to_label = map_clusters_to_labels(y_test, y_test_cluster, n_components=3)

In [23]:
scores = evaluate(y_test, y_test_pred)
print("Đánh giá kết quả trên tập test:")
for label, metric in scores.items():
    print(f"Sentiment {label}: Precision={metric['precision']:.3f}, Recall={metric['recall']:.3f}, F1={metric['f1']:.3f}")

Đánh giá kết quả trên tập test:
Sentiment negative: Precision=0.510, Recall=0.421, F1=0.461
Sentiment neutral: Precision=0.000, Recall=0.000, F1=0.000
Sentiment positive: Precision=0.536, Recall=0.676, F1=0.598


In [25]:
def count_labels(y):
    unique_labels, counts = np.unique(y, return_counts=True)
    for label, count in zip(unique_labels, counts):
        print(f"{label}: {count}")

count_labels(y_test)

negative: 1409
neutral: 167
positive: 1590


In [26]:
print(cluster_to_label)

{0: np.str_('positive'), 1: np.str_('negative'), 2: np.str_('positive')}


In [28]:
def analyze_cluster_distribution(y_true, y_pred, n_components):
    """
    Phân tích phân bố nhãn thật trong từng cluster.
    """
    print("📊 Phân bố nhãn thật trong từng cụm:")
    for c in range(n_components):
        indices = np.where(y_pred == c)[0]
        if len(indices) == 0:
            print(f"Cluster {c}: (trống)")
            continue
        true_labels = y_true[indices]
        unique_labels, counts = np.unique(true_labels, return_counts=True)
        print(f"\nCluster {c} ({len(indices)} điểm):")
        for label, count in zip(unique_labels, counts):
            percent = (count / len(indices)) * 100
            print(f"  - {label}: {count} ({percent:.2f}%)")

analyze_cluster_distribution(y_test, y_test_cluster, n_components=3)

📊 Phân bố nhãn thật trong từng cụm:

Cluster 0 (1 điểm):
  - positive: 1 (100.00%)

Cluster 1 (1162 điểm):
  - negative: 593 (51.03%)
  - neutral: 54 (4.65%)
  - positive: 515 (44.32%)

Cluster 2 (2003 điểm):
  - negative: 816 (40.74%)
  - neutral: 113 (5.64%)
  - positive: 1074 (53.62%)


### Nhận xét
- Sau khi đánh giá kết quả trên tập test, đối với nhãn negative, mô hình phân cụm cho nhãn này khá là nửa vời, chỉ gán đúng gần như là 1 nửa, còn lại là gán nhầm các mẫu dữ liệu của cụm khác, đối với nhãn positive thì cũng gần như tương tự nhưng tốt hơn 1 tí ở chỗ khả năng tìm được đúng nhãn positive cao hơn, đặc biệt là nhãn neutral, mô hình không nhận diện được bất kỳ mẫu nào thuộc nhãn này.
- Xem phân bố nhãn trong tập test thì có thể thấy được là mẫu dữ liệu có nhãn neutral là cực kỳ ít, tiếp theo quan sát kết quả ánh xạ từ cụm sang nhãn (ánh xạ đơn giản bằng cách đếm số nhãn xuất hiện nhiều nhất của cụm đó và map cụm cho tên của nhãn đó) thì thấy là có tới 2 cụm được gán là positive, không có cụm nào đại diện cho neutral. Quan sát tiếp phân bố nhãn thật trong từng cụm, có thể thấy rõ hơn phân bố của nhãn neutral, không có cụm chuyên biệt riêng cho nhãn này nên là bị trộn lẫn bởi những cụm khác.
- Nguyên nhân dẫn đến việc đánh giá kém trên tập test, đặc biệt là kém trên nhãn neutral là vì dữ liệu bị mất cân bằng, GMM là một mô hình thuộc loại không giám sát nên nhiệm vụ của nó chỉ là phân cụm nên khi ánh xạ sang label, neutral sẽ không bao giờ được chọn vì nó quá ít. Ngoài ra, nguyên nhân cũng có thể đến từ việc embedding của PhoBERT đã không tốt cho neutral ngay từ đầu, giảm chiều chưa giữ lại đủ thông tin, ...
- Kết luận là mô hình GMM hoạt động "kha khá" ổn đối với nhãn positive và negative, rất kém đối với nhãn neutral.